In [1]:
import AdaCapNet
seed = 0

# Get boston dataset

In [2]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split as tts
from sklearn.datasets import load_boston as lb
def prepare_dataset(X, y, train_size = 0.8, normalize_X = True, seed= False):
    kwargs = {}
    if seed or type(seed)== type(0):
        kwargs["random_state"] = seed
        np.random.seed(seed)
    if normalize_X: X = StandardScaler().fit_transform(X)
    X_train, X_test, y_train, y_test = tts(X, y, train_size = train_size, **kwargs)
    return X_train, X_test, y_train, y_test
def prepare_boston(train_size = 0.8, seed= 0, normalize_X = True, standardize_y = True):
    X,y = lb(return_X_y=True)
    if standardize_y: y = (y - y.mean())/y.std()
    X_train, X_test, y_train, y_test = prepare_dataset(X, y, train_size = train_size , seed=seed, normalize_X = normalize_X)
    return X_train, X_test, y_train, y_test

In [3]:
X_train, X_test, y_train, y_test = prepare_boston(seed = seed, standardize_y = True)

# Pick FFNN architecture

## simple MLP

In [4]:
import architectures
architecture = architectures.DenseLayers
architecture_parameters =  {"width":512,"depth":2,"n_features":X_train.shape[1]}
architecture_parameters["output"] = 1 #regression

## duplicate but without output layer

In [5]:
from copy import deepcopy
hidden_layers_parameters = deepcopy(architecture_parameters)
hidden_layers_parameters["output"] = 0

# Evaluate RF, MLP, AdaCap

## Check Random Forest Performance

In [6]:
from sklearn.ensemble import RandomForestRegressor as RF
reg = RF(n_estimators = 1000, random_state = seed)
reg.fit(X_train,y_train)
print("RF","R2-score :", reg.score(X_test, y_test))
del reg

RF R2-score : 0.7687240238093025


## Init Fit Predict simple MLP

In [7]:
use_AdaCap = False
reg = AdaCapNet.AdaCapNetRegressor(hidden_nn = architecture, hidden_params = architecture_parameters, closeform_parameter_init = use_AdaCap, random_state = seed, )
reg.fit(X_train,y_train)
print("MLP:","R2-score :",reg.score(X_test, y_test))
reg.delete_model_weights() #else gpu memory leaks
del reg

MLP: R2-score : 0.7669409780204398


## Init Fit Predict simple AdaCapMLP

In [8]:
use_AdaCap = True
reg = AdaCapNet.AdaCapNetRegressor(hidden_nn = architecture, hidden_params = hidden_layers_parameters, closeform_parameter_init = "max_variation", random_state = seed)
reg.fit(X_train,y_train)
print("AdaCapMLP:","R2-score :",reg.score(X_test, y_test))
reg.delete_model_weights() #else gpu memory leaks
del reg

AdaCapMLP: R2-score : 0.7803205151159722


# Pick architecture and parameters from benchmarked architectures

In [9]:
from benchmarked_architectures import *
parameter_set = deepcopy(eval("adacapnetresblock"))
parameter_set["hidden_params"]["n_features"] = X_train.shape[1] #you always need to specify this information in advance

In [10]:
use_AdaCap = True
reg = AdaCapNet.AdaCapNetRegressor(random_state = seed, **parameter_set)
reg.fit(X_train,y_train)
print("AdaCapResblock:","R2-score :",reg.score(X_test, y_test))
reg.delete_model_weights() #else gpu memory leaks
del reg

AdaCapResblock: R2-score : 0.805933525040707
